In [2]:
import sunpy.data.sample
from astropy.visualization import ImageNormalize, SqrtStretch
from skimage.transform import rescale, AffineTransform, warp
from skimage.metrics import structural_similarity as ssim
from astropy.coordinates import EarthLocation, SkyCoord
from datetime import date, datetime, timedelta
import sunpy.visualization.colormaps as cm
from sunpy.net import Fido, attrs as SAF
from scipy.stats.stats import pearsonr
#from iris_lmsalpy import hcr2fits
import matplotlib.pyplot as plt
import scipy.ndimage as scnd
from astropy.io import fits
import astropy.units as u
from tqdm import tqdm
import numpy as np
import matplotlib
import sunpy.map
import warnings
import salat
import glob
import os
warnings.filterwarnings("ignore")

/tmp/ipykernel_794795/1882375747.py:9: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [3]:
path_ALMA = '/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/ALMA_data/{}'
file = path_ALMA.format(
    'D06_solaralma.b3.fba.20180412_155228-162441.2017.1.00653.S.level4.k.fits')
almacube, header, timesec, timeutc, beammajor, beamminor, beamangle = salat.read(file, timeout=True, beamout=True,
                                                                                 HEADER=True, SILENT=False, fillNan=True)


---------------------------------------------------
--------------- SALAT READ part of ----------------
-- Solar Alma Library of Auxiliary Tools (SALAT) --

Reading ALMA cube



100%|█████████████████████████████████████| 1608/1608 [00:01<00:00, 1211.43it/s]


Do you want to fill NaN with data median? (y/n): Y
---------------------------------------------------
--------------- SALAT INFO part of ----------------
-- Solar Alma Library of Auxiliary Tools (SALAT) --


----------------------------------------------
| Data feat.: 
----------------------------------------------
|  ALMA BAND:  3
|  Obs. Date:  2018-04-12
|  ALMA proj:  2017.1.00653.S
|  Pix. Unit:  K
|  Pix. Size:  0.3  arcsec.
|  Beam mean:  2.15798  arcsec
|  FOV. diam:  notcomputed
----------------------------------------------
| Data range 
----------------------------------------------
|  Min =  5260.81054688  Kelvin
|  Max =  10677.8203125  Kelvin
----------------------------------------------

Done!


In [4]:
def aperture_circ_ALMA(alma_cube):
    """
    This function allow us to do a mask of a matrix with the FoV of ALMA

    Input  : almacube obtained using SALSA 
    Output : A matrix mask with the same shape to ALMA'S IMAGES 
    """
    x_size = alma_cube.shape[1]
    n = x_size
    # total size of the grid  in meters
    a = x_size
    aperture = np.zeros([n, n])
    # x grid 1D
    x = np.linspace(-a/2.0, a/2.0, n)
    # y grid 1D
    y = np.linspace(-a/2.0, a/2.0, n)
    # radius of the telescope
    R = x_size/2
    # making  circular aperture
    for j in range(0, n, 1):
        for i in range(0, n, 1):
            if ((x[i]**2.0 + y[j]**2.0)**(1/2) <= R):
                aperture[i, j] = 1.0
    return aperture


def fun_alma_rotate(cubo_alma, header_alma):
    """
    Regard to IRIS/SDO images the ALMA images are rotated. 
    This function delete this rotation and applied  the 
    function aperture_circ_ALMA to has the FoV of ALMA
    
    --------------------Input----------------------------------
    Parameters: cubo_alma : type array taken from  SALSA  
                           is a vector where its position correspond a matrices 
                           (A photogram of  the sum)  
                header:
                Header of datacube where the angle of rotation will be taken  
    Output: 
                ALMA_CUBE without rotation  
    """
    # rotating ALMDA DATA
    imag_alma_rot = []
    print('Rotating ALMA cube')
    for i in tqdm(range(0, len(cubo_alma), 1), colour='green'):
        imag_alma_rot.append(scnd.interpolation.rotate(cubo_alma[i],
                                                       angle=header_alma['SOLAR_P'], reshape=False))
    print('rotate angle', header_alma['SOLAR_P'])
    aperture = aperture_circ_ALMA(cubo_alma)
    imag_alma_rot = imag_alma_rot*aperture
    # cubo_alma[0][0][0] has mean value from ALMA cube
    imag_alma_rot[imag_alma_rot < 0.05] = cubo_alma[0][0][0]
    return imag_alma_rot


def find_nearest(array, alma_t0, alma_tf):
    '''
    This function allows us to find (in a time vector) 
    the initial and final times at times alma_t0, alma_tf.
    
    --------------------Input----------------------------------
    Parameters: array : type array and its elements is in the datetime.datetime 
                        format (in this case we  obtained this vector using the )
                        in our case this time vector was obtained with the DATA-OBS 
                        of each previously downloaded SDO frame  
                alma_t0: type T0 is  where the ALMA observation has started
                         datetime.datetime  format
                alma_tf: type Tf is  where the ALMA observation has finalized 
                         datetime.datetime  format  
    --------------------Output----------------------------------
    Positions in the vector array (array) closest to the   alma_t0, alma_tf        
    '''
    array = np.asarray(array)
    idx_0 = (np.abs(array - alma_t0)).argmin()
    idx_f = (np.abs(array - alma_tf)).argmin()
    print('______________________________________________________')
    print('t_0_SDO =', array[idx_0])
    print('t_0_ALMA=', alma_t0)
    print('______________________________________________________')
    print('t_f_SDO =', array[idx_f])
    print('t_0_ALMA=', alma_tf)
    print('______________________________________________________')
    print('posicion_n_0:', idx_0, '| posicion_n_f:', idx_f)
    print('______________________________________________________')
    return idx_0, idx_f

## Rotating ALMA cube

In [6]:
# Rotating ALMA cube
alma_rote = fun_alma_rotate(almacube, header)

Rotating ALMA cube


100%|██████████████████████████████████████| 1608/1608 [00:10<00:00, 152.63it/s]


rotate angle -26.1446


In [7]:
def sdo_align_ima(sdo_file, time_filter, time_alma, alma_cube, alma_resolution, path_mapMap):

    '''
    This function allow us to estimate the helioprojetive coordinate 
    to who the SDO and ALMA images are 'align '. For that, we use pearson 
    and SSIM  (Structural similarity) coefficient
    ----------------------------Input----------------------------------
    Parameters: 
                sdo_file:    type array, it is a vector (datacube) where   its elements is
                             correspond to a matrices (photos)  taken by SDO/AIA or HMI
                             In our case, the photos we used were those of the SDO cropped 
                             to a slightly larger FoV than the soul to be able to align the data
                time_filter: type array and its elements is in the datetime.datetime 
                             format in our case this time vector was obtained with the DATA-OBS 
                             of each previously downloaded SDO frame
                time_alma:   type array and its elements is in the datetime.datetime 
                             format. In this case we  obtained this vector from SALSA database

                alma_cube:   it is a vector (datacube) where   its elements is
                             correspond to a matrices (photos)  taken by ALMA,
                             In this case we  obtained this vector from SALSA database
                alma_resolution:  Taken from the header or ALMA database

                path_mapMap: type  str, corresponds to the path where the information of  
                             the SDO snippets loaded in the sdo_file variable was saved. This path 
                             is necessary to extract the information of the coordinates of the (HEADER)
    -------------------------Output----------------------------------------
    Helioprotective coordinate for which the maximum correlation is found between the SDO and ALMA data 
    for both alignment methods (Pearson and SSIM).  Also, it shows the 'r' of Pearson and SSIM, scale 
    factor of the images (the SDO images were taken to the ALMA resolution)

    '''

    url = sorted(glob.glob(path_mapMap))

    
    #name_filter= 'SDO/AIA {}'.format(94)
    
    try:
        name_filter = fits.open(url[0])[0].header['TELESCOP']+ \
                  str('-')+fits.open(url[0])[0].header['WAVE_STR']
        print('|-----------------{}-------------------|'.format(name_filter))  
      
    except:
        name_filter = fits.open(url[0])[0].header['TELESCOP']+ \
              str('-')+fits.open(url[0])[0].header['INSTRUME']
        print('|-----------------{}-------------------|'.format(name_filter)) 

    #name_filter = fits.open(url[0])[0].header['TELESCOP']+ \
    #              str('-')+fits.open(url[0])[0].header['WAVE_STR']
    #print('|-----------------{}-------------------|'.format(name_filter))
    
    # To scale IRIS resolution
    sdo_resolution = fits.open(url[0])[0].header['CDELT1']
    cadence_filter = time_filter[1]-time_filter[0]
    scale_factor = sdo_resolution/alma_resolution
    print('scale_factor:', scale_factor)
    print('Rescaling AIA  images')
    # For example to AIA 94A filter
    # from (226, 152, 151) ----> (950, 429, 408)
    # same resolution to SDO/AIA AND HMI-ALMA 0.3
    rescaled_sdo = []
    for i in tqdm(range(len(sdo_file)), colour='green'):
        rescaled_sdo.append(rescale(sdo_file[i], scale_factor, order=0,
                                    anti_aliasing=False, preserve_range=True))
    rescaled_sdo = np.array(rescaled_sdo)

    print('________________________________________________________________________')
    print('initial shape:', sdo_file.shape,
          '| rescaled shape:', rescaled_sdo.shape)
    # --------------Selecting the same windown of ALMA----------------
    posicion_n_0, posicion_n_f = find_nearest(
        time_filter, time_alma[0], time_alma[-1])

    # -------------- Mean IRIS and ALMA datacube-----------------------
    mean_sdo    = np.sum([i for i in rescaled_sdo[int(posicion_n_0):int(
        posicion_n_f)]], 0)/len(rescaled_sdo[int(posicion_n_0):int(posicion_n_f)])
    mean_alma   = np.sum([i for i in alma_cube], 0)/len(alma_cube)
    sdo_interp  = np.interp(mean_sdo, (np.percentile(
        mean_sdo, 11), np.percentile(mean_sdo, 95)), (0, +1))
    alma_interp = np.interp(mean_alma, (np.percentile(
        mean_alma, 11), np.percentile(mean_alma, 95)), (0, +1))
    # -------------- ALMA_VECTOR ---------------------------------------
    flatten_alma = alma_interp.flatten()
    ccrr_pearson = np.zeros((sdo_interp.shape[0], sdo_interp.shape[1]))
    ccrr_ssim    = np.zeros((sdo_interp.shape[0], sdo_interp.shape[1]))

    print('|----Estimating Pearson and SSIM coeffients---- | ')
    for i in tqdm(range(0, sdo_interp.shape[0]-alma_interp.shape[0]+1, 1), position=0, desc="i", leave=False, colour='green', ncols=100):
        for j in range(0, sdo_interp.shape[1] - alma_interp.shape[1]+1, 1):
            matrix = sdo_interp[0+i:alma_interp.shape[0]+i, 0+j:alma_interp.shape[1]+j].copy()
            corr, valor_p = pearsonr(matrix.flatten(), flatten_alma)
            ssim_none = ssim(matrix, alma_interp,
                             data_range=sdo_interp.max() - sdo_interp.min())
            x = i+alma_interp.shape[1]//2
            y = j+alma_interp.shape[0]//2
            ccrr_pearson[x][y] = corr
            ccrr_ssim[x][y]    = ssim_none

    x_max_pearson = np.where(ccrr_pearson == np.max(ccrr_pearson))[1][0]
    y_max_pearson = np.where(ccrr_pearson == np.max(ccrr_pearson))[0][0]
    x_max_ssim    = np.where(ccrr_ssim    == np.max(ccrr_ssim))[1][0]
    y_max_ssim    = np.where(ccrr_ssim    == np.max(ccrr_ssim))[0][0]


    print('________________________________________________________________________')
    print('x_max_pearson=', x_max_pearson, '| y_max_pearson=',
          y_max_pearson, "| r=", np.max(ccrr_pearson))


    print('________________________________________________________________________')
    print('x_max_ssim   =', x_max_ssim, '| y_max_ssim   =',
          y_max_ssim,   "| r=", np.max(ccrr_ssim))

    print('________________________________________________________________________')
    print('|-----Helioprojective Coordinates-------|')
    map_sunpy                            = sunpy.map.Map(url[posicion_n_0])
    x_scaled_pearson , y_scaled_pearson  = x_max_pearson/scale_factor   ,  y_max_pearson/scale_factor
    cord_pearson                         = map_sunpy.pixel_to_world(x_scaled_pearson *u.pix,  y_scaled_pearson *u.pix )

    print('Pearson: (Tx, Ty) in arcsec',cord_pearson.Tx.value,cord_pearson.Ty.value)

    x_scaled_ssim, y_scaled_ssim =  x_max_ssim/scale_factor   ,  y_max_ssim/scale_factor
    cord_ssim               = map_sunpy.pixel_to_world(x_scaled_ssim*u.pix,  y_scaled_ssim*u.pix )
    print('SSIM   : (Tx, Ty) in arcsec',cord_ssim.Tx.value,cord_ssim.Ty.value)
    print('________________________________________________________________________')
    print('________________________________________________________________________')
    print('________________________________________________________________________')
    



In [8]:
'''
path_sdo = '/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_data_cubic/{}'
cubo_171 = np.load(path_sdo.format('cuboAIA171.npy'), allow_pickle=True).astype(np.float32)                   
time_171 = np.load(path_sdo.format('cuboAIA_timeUTC_171.npy'),  allow_pickle=True)
wl = [94, 131, 171, 193, 211, 304, 335, 1600, 1700]
path_crop ='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_COMPLETOS/{}/crop/*.fits'.format(
    wl[2])
sdo_align_ima(cubo_171, time_171, timeutc, alma_rote, header['CDELT1A'] ,path_crop)
'''

##   Aligning ALMA with SDO without rotating the SDO crops

In [7]:
#wl = [94]
wl = [94, 131, 171, 193, 211, 304, 335, 1600, 1700]

for i in wl:
    #--------------Loading data cube in forma npy for every AIA filter------------------------------
    path_cube='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_data_cubic'+ \
              '/cuboAIA{filter}.npy'.format(filter=i)
    path_time='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_data_cubic' + \
              '/cuboAIA_timeUTC_{filter}.npy'.format(filter=i)
    cube_data_load=  np.load(path_cube, allow_pickle=True).astype(np.float32) 
    cube_time_load=  np.load(path_time, allow_pickle=True)
    #--------------Loading crops in .fits  format ------------------------------
    path_crop ='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_COMPLETOS'+ \
               '/{}/crop/*.fits'.format(i) 
    #---------------Using function sdo_align_ima ----------------------
    sdo_align_ima(cube_data_load, cube_time_load, timeutc, alma_rote, header['CDELT1A'],path_crop)

|-----------------SDO/AIA-94_THIN-------------------|
scale_factor: 2.0003633333333335
Rescaling AIA  images


100%|██████████| 226/226 [00:00<00:00, 446.09it/s]


________________________________________________________________________
initial shape: (226, 191, 191) | rescaled shape: (226, 382, 382)
______________________________________________________
t_0_SDO = 2018-04-12 15:52:35.120000
t_0_ALMA= 2018-04-12 15:52:31.992187
______________________________________________________
t_f_SDO = 2018-04-12 16:24:35.120000
t_0_ALMA= 2018-04-12 16:24:40.246093
______________________________________________________
posicion_n_0: 35 | posicion_n_f: 195
______________________________________________________
|----Estimating Pearson and SSIM coeffients---- | 


________________________________________________________________________
x_max_pearson= 224 | y_max_pearson= 255 | r= 0.17013104662240985
________________________________________________________________________
x_max_ssim   = 189 | y_max_ssim   = 238 | r= 0.02790817616257489
________________________________________________________________________
|-----Helioprojective Coordinates-------|
Pearson: (Tx, Ty) in arcsec -138.2581859617494 -419.2446979798585
SSIM   : (Tx, Ty) in arcsec -127.74596959236078 -414.16997088115625
|-----------------SDO/AIA-131_THIN-------------------|
scale_factor: 2.002326666666667
Rescaling AIA  images


100%|██████████| 226/226 [00:00<00:00, 511.44it/s]


...

________________________________________________________________________
x_max_pearson= 223 | y_max_pearson= 233 | r= 0.24708761071957586
________________________________________________________________________
x_max_ssim   = 228 | y_max_ssim   = 230 | r= 0.06547243624731396
________________________________________________________________________
|-----Helioprojective Coordinates-------|
Pearson: (Tx, Ty) in arcsec -137.9019109455403 -413.13964674101004
SSIM   : (Tx, Ty) in arcsec -139.40222033229657 -412.2401655034645
|-----------------SDO/AIA-1700-------------------|
scale_factor: 2.0429933333333334
Rescaling AIA  images


100%|██████████| 111/111 [00:00<00:00, 547.10it/s]


________________________________________________________________________
initial shape: (111, 187, 187) | rescaled shape: (111, 382, 382)
______________________________________________________
t_0_SDO = 2018-04-12 15:52:28.720000
t_0_ALMA= 2018-04-12 15:52:31.992187
______________________________________________________
t_f_SDO = 2018-04-12 16:24:28.720000
t_0_ALMA= 2018-04-12 16:24:40.246093
______________________________________________________
posicion_n_0: 17 | posicion_n_f: 96
______________________________________________________
|----Estimating Pearson and SSIM coeffients---- | 


                                                                                                    1.32s/it]

________________________________________________________________________
x_max_pearson= 222 | y_max_pearson= 233 | r= 0.2558681987945488
________________________________________________________________________
x_max_ssim   = 228 | y_max_ssim   = 229 | r= 0.05851818581733749
________________________________________________________________________
|-----Helioprojective Coordinates-------|
Pearson: (Tx, Ty) in arcsec -137.81181484134868 -413.0551749431002
SSIM   : (Tx, Ty) in arcsec -139.61224343022332 -411.8558219435727


In [67]:
path_cube_hmi_crop='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_data_cubic'+ \
                   '/cuboHMI_MAGT.npy'
path_time_hmi_crop='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_data_cubic'+ \
                   '/cuboHMI_MAGT_timeUTC.npy'
path_hmi_crops='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_COMPLETOS'+ \
               '/{}/crop/*.fits'.format('magt') 
cube_data_hmi=  np.load(path_cube_hmi_crop, allow_pickle=True) 
cube_time_hmi=  np.load(path_time_hmi_crop, allow_pickle=True)
sdo_align_ima(cube_data_hmi, cube_time_hmi, timeutc, alma_rote, header['CDELT1A'],path_hmi_crops)

|-----------------SDO/HMI-HMI_FRONT2-------------------|
scale_factor: 1.6811633333333336
Rescaling AIA  images


100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 413.39it/s]

________________________________________________________________________
initial shape: (60,) | rescaled shape: (60,)
______________________________________________________
t_0_SDO = 2018-04-12 15:52:16.700001
t_0_ALMA= 2018-04-12 15:52:31.992187
______________________________________________________
t_f_SDO = 2018-04-12 16:24:31.700001
t_0_ALMA= 2018-04-12 16:24:40.246093
______________________________________________________
posicion_n_0: 9 | posicion_n_f: 52
______________________________________________________


ValueError: operands could not be broadcast together with shapes (382,382) (383,382) 

## Aligning ALMA with SDO rotating  the entire solar disk and crops 

In [8]:
#wl = [94]
wl = [94, 131, 171, 193, 211, 304, 335, 1600, 1700]

for i in wl:
    #--------------Loading data cube in forma npy for every AIA filter------------------------------
    path_cube='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_data_cubic_rotate_and_crop'+ \
              '/rotate_all_disk_and_crop_AIA_{filter}.npy'.format(filter=i)
    # The time vector  not affected by rotation
    path_time='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_data_cubic' + \
              '/cuboAIA_timeUTC_{filter}.npy'.format(filter=i)
    cube_data_load=  np.load(path_cube, allow_pickle=True).astype(np.float32) 
    cube_time_load=  np.load(path_time, allow_pickle=True)
    #--------------Loading crops in .fits  format ------------------------------
    path_crop ='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_COMPLETOS'+ \
               '/{}/rotate_all_disk_and_crop/*.fits'.format(i) 
    #---------------Using function sdo_align_ima ----------------------
    sdo_align_ima(cube_data_load, cube_time_load, timeutc, alma_rote, header['CDELT1A'],path_crop)

|-----------------SDO/AIA-94_THIN-------------------|
scale_factor: 2.0003633333333335
Rescaling AIA  images


100%|██████████| 226/226 [00:00<00:00, 519.42it/s]


________________________________________________________________________
initial shape: (226, 191, 190) | rescaled shape: (226, 382, 380)
______________________________________________________
t_0_SDO = 2018-04-12 15:52:35.120000
t_0_ALMA= 2018-04-12 15:52:31.992187
______________________________________________________
t_f_SDO = 2018-04-12 16:24:35.120000
t_0_ALMA= 2018-04-12 16:24:40.246093
______________________________________________________
posicion_n_0: 35 | posicion_n_f: 195
______________________________________________________
|----Estimating Pearson and SSIM coeffients---- | 


________________________________________________________________________
x_max_pearson= 176 | y_max_pearson= 192 | r= 0.20634821809915121
________________________________________________________________________
x_max_ssim   = 173 | y_max_ssim   = 189 | r= 0.02836686535108312
________________________________________________________________________
|-----Helioprojective Coordinates-------|
Pearson: (Tx, Ty) in arcsec -131.92371852439828 -399.5262860138532
SSIM   : (Tx, Ty) in arcsec -132.82371815363877 -400.42628132421066
|-----------------SDO/AIA-131_THIN-------------------|
scale_factor: 2.002326666666667
Rescaling AIA  images


100%|██████████| 226/226 [00:00<00:00, 549.33it/s]


...

________________________________________________________________________
x_max_pearson= 181 | y_max_pearson= 201 | r= 0.5121977971544621
________________________________________________________________________
x_max_ssim   = 181 | y_max_ssim   = 200 | r= 0.11797204444871022
________________________________________________________________________
|-----Helioprojective Coordinates-------|
Pearson: (Tx, Ty) in arcsec -130.79667650396004 -396.8439279193414
SSIM   : (Tx, Ty) in arcsec -130.79667650396004 -397.143926747753
|-----------------SDO/AIA-1700-------------------|
scale_factor: 2.0429933333333334
Rescaling AIA  images


100%|██████████| 111/111 [00:00<00:00, 262.42it/s]


________________________________________________________________________
initial shape: (111, 187, 186) | rescaled shape: (111, 382, 380)
______________________________________________________
t_0_SDO = 2018-04-12 15:52:28.720000
t_0_ALMA= 2018-04-12 15:52:31.992187
______________________________________________________
t_f_SDO = 2018-04-12 16:24:28.720000
t_0_ALMA= 2018-04-12 16:24:40.246093
______________________________________________________
posicion_n_0: 17 | posicion_n_f: 96
______________________________________________________
|----Estimating Pearson and SSIM coeffients---- | 


                                                                                                    2.06s/it]

________________________________________________________________________
x_max_pearson= 179 | y_max_pearson= 201 | r= 0.47558031154074
________________________________________________________________________
x_max_ssim   = 179 | y_max_ssim   = 200 | r= 0.09812801716708394
________________________________________________________________________
|-----Helioprojective Coordinates-------|
Pearson: (Tx, Ty) in arcsec -131.04505892260931 -396.80359219845514
SSIM   : (Tx, Ty) in arcsec -131.04505892260931 -397.10359102681434


##  Aligning ALMA with SDO croping and then rotated

In [8]:
wl = [304]
#wl = [94, 131, 171, 193, 211, 304, 335, 1600, 1700]

for i in wl:
    #--------------Loading data cube in forma npy for every AIA filter------------------------------
    path_cube='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_data_cubic_crop_and_rotate'+ \
              '/cubo_crop_and_rotate_AIA_{filter}.npy'.format(filter=i)
    # The time vector  not affected by rotation
    path_time='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_data_cubic' + \
              '/cuboAIA_timeUTC_{filter}.npy'.format(filter=i)
    cube_data_load=  np.load(path_cube, allow_pickle=True).astype(np.float32) 
    cube_time_load=  np.load(path_time, allow_pickle=True)
    #--------------Loading crops in .fits  format ------------------------------
    path_crop ='/media/javier/SSD_2/D06_region/datos_SDO-completos_IRIS_ALMA/SDO_COMPLETOS'+ \
               '/{}/crop_and_rotate/*.fits'.format(i) 
    #---------------Using function sdo_align_ima ----------------------
    sdo_align_ima(cube_data_load, cube_time_load, timeutc, alma_rote, header['CDELT1A'],path_crop)

|-----------------SDO/AIA-304_THIN-------------------|
scale_factor: 2.00055
Rescaling AIA  images


100%|████████████████████████████████████████| 225/225 [00:00<00:00, 609.21it/s]


________________________________________________________________________
initial shape: (225, 193, 193) | rescaled shape: (225, 386, 386)
______________________________________________________
t_0_SDO = 2018-04-12 15:52:29.140000
t_0_ALMA= 2018-04-12 15:52:31.992187
______________________________________________________
t_f_SDO = 2018-04-12 16:24:41.120000
t_0_ALMA= 2018-04-12 16:24:40.246093
______________________________________________________
posicion_n_0: 35 | posicion_n_f: 196
______________________________________________________
|----Estimating Pearson and SSIM coeffients---- | 


                                                                                                    1.00it/s]

________________________________________________________________________
x_max_pearson= 173 | y_max_pearson= 200 | r= 0.5433225107691669
________________________________________________________________________
x_max_ssim   = 169 | y_max_ssim   = 201 | r= 0.23257274117278373
________________________________________________________________________
|-----Helioprojective Coordinates-------|
Pearson: (Tx, Ty) in arcsec -133.93086438719183 -397.4334329506414
SSIM   : (Tx, Ty) in arcsec -135.13086387654766 -397.13343261983226
________________________________________________________________________
________________________________________________________________________
________________________________________________________________________
